In [2]:
library(Seurat)

Loading required package: SeuratObject

Loading required package: sp

'SeuratObject' was built with package 'Matrix' 1.7.0 but the current
version is 1.7.1; it is recomended that you reinstall 'SeuratObject' as
the ABI for 'Matrix' may have changed


Attaching package: 'SeuratObject'


The following objects are masked from 'package:base':

    intersect, t




In [6]:
data_dir <- "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05"
dirs <- list.dirs(path = data_dir, full.names = TRUE, recursive =FALSE)
print(dirs)

 [1] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000001"
 [2] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000002"
 [3] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000003"
 [4] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000004"
 [5] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000005"
 [6] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000006"
 [7] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000007"
 [8] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000008"
 [9] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000009"
[10] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000010"
[11] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000011"
[12] "./in/SCPCP000001_SINGLE-CELL_SINGLE-CELL-EXPERIMENT_2024-12-05/SCPCS000012"
[13] "./in/SCPCP

In [5]:
getwd()

[1] "/Users/shiwani/Documents/GITHUB/Pediatric_Immuno-Oncology/Notebooks/Generate_HGG_reference"

In [12]:
seurat_objects <- list()

for (dir_name in dirs) {
    sample_name <- basename(dir_name)
    modified_sample_name <- sub("SCPCS", "SCPCL", sample_name)
    filename <- paste0(dir_name, "/", modified_sample_name, "_processed.rds")
    sce <- readRDS(filename)
    seurat_obj <- as.Seurat(sce, counts = "counts", data=NULL)
    seurat_objects[[modified_sample_name]] <- seurat_obj
}

Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from PC to PC_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from UMAP to UMAP_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from PC to PC_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from UMAP to UMAP_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from PC to PC_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from UMAP to UMAP_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from PC to PC_"
Warning message:
"Keys should be one or more alphanumeric characters followed by an underscore, setting key from UMAP to

In [13]:
library(harmony)

Warning message:
"package 'harmony' was built under R version 4.4.1"
Loading required package: Rcpp

Warning message:
"package 'Rcpp' was built under R version 4.4.1"


In [15]:
for (name in names(seurat_objects)) {
    cat(name, unique(seurat_objects[[name]]@meta.data$orig.identis), nrow(seurat_objects[[name]]@meta.data), "\n")
}

SCPCL000001 2623 
SCPCL000002 1016 
SCPCL000003 278 
SCPCL000004 546 
SCPCL000005 326 
SCPCL000006 1839 
SCPCL000007 1324 
SCPCL000008 3720 
SCPCL000009 644 
SCPCL000010 199 
SCPCL000011 742 
SCPCL000012 1025 
SCPCL000013 2956 
SCPCL000014 766 
SCPCL000015 1152 
SCPCL000016 2799 
SCPCL000017 773 
SCPCL000018 39 
SCPCL000019 1558 
SCPCL000020 411 
SCPCL000021 827 
SCPCL000022 452 
SCPCL000023 1290 


In [16]:
head(rownames(seurat_objects[[1]]))

[1] "ENSG00000223972" "ENSG00000243485" "ENSG00000284332" "ENSG00000268020"
[5] "ENSG00000240361" "ENSG00000186092"

In [17]:
library(AnnotationDbi)
library(org.Hs.eg.db)  # Change to org.Mm.eg.db for Mouse
library(dplyr)




Attaching package: 'dplyr'


The following object is masked from 'package:AnnotationDbi':

    select


The following object is masked from 'package:Biobase':

    combine


The following objects are masked from 'package:GenomicRanges':

    intersect, setdiff, union


The following object is masked from 'package:GenomeInfoDb':

    intersect


The following objects are masked from 'package:IRanges':

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from 'package:S4Vectors':

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from 'package:BiocGenerics':

    combine, intersect, setdiff, union


The following object is masked from 'package:matrixStats':

    count


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [18]:
convert_ensembl_to_symbol <- function(ensembl_ids, species = "hsapiens") {
  # Select the appropriate OrgDb based on species
  if (species == "hsapiens") {
    orgdb <- org.Hs.eg.db
  } else if (species == "mmusculus") {
    orgdb <- org.Mm.eg.db
  } else {
    stop("Species not supported. Please use 'hsapiens' or 'mmusculus'.")
  }
  
  # Remove version numbers from Ensembl IDs (e.g., ENSG000001234.5 -> ENSG000001234)
  ensembl_ids_clean <- sub("\\..*", "", ensembl_ids)
  
  # Map Ensembl IDs to gene symbols
  gene_symbols <- mapIds(
    orgdb,
    keys = ensembl_ids_clean,
    column = "SYMBOL",
    keytype = "ENSEMBL",
    multiVals = "first"  # Use the first symbol if multiple are found
  )
  
  return(gene_symbols)
}

In [ ]:

# Iterate through each Seurat object in selected_list
for (name in names(seurat_objects)) {
  seurat_obj <- seurat_objects[[name]]
  
  # Get current gene identifiers
  genes <- rownames(seurat_obj)
  
  # Check if genes are Ensembl IDs (assuming Ensembl IDs start with "ENSG")
  if (any(grepl("^ENSG", genes))) {
    message(paste("Converting Ensembl IDs to gene symbols in", name))
    
    # Convert Ensembl IDs to gene symbols
    gene_symbols <- convert_ensembl_to_symbol(genes, species = "hsapiens")  # Change species if needed
    
    # Identify genes with valid gene symbols
    valid_genes <- !is.na(gene_symbols) & gene_symbols != ""
    
    if (sum(valid_genes) == 0) {
      warning(paste("No valid gene symbols found for", name, ". Skipping this object."))
      next
    }
    
    # Subset Seurat object to keep only valid genes
    seurat_obj_subset <- subset(seurat_obj, features = genes[valid_genes])
    gene_symbols_subset <- gene_symbols[valid_genes]
    
    # Get the counts matrix
    counts_matrix <- GetAssayData(seurat_obj_subset, slot = "counts")
    
    # Replace rownames with gene symbols
    rownames(counts_matrix) <- gene_symbols_subset
    
    # Handle duplicate gene symbols by summing their counts
    # Convert to data frame for manipulation
    counts_df <- as.data.frame(counts_matrix)
    counts_df$gene_symbol <- rownames(counts_matrix)
    
    # Aggregate counts by gene symbol
    counts_aggregated <- counts_df %>%
      group_by(gene_symbol) %>%
      summarise(across(everything(), sum)) %>%
      ungroup()
    
    # Convert back to a matrix
    counts_matrix_aggregated <- as.matrix(counts_aggregated[,-1])
    rownames(counts_matrix_aggregated) <- counts_aggregated$gene_symbol
    
    # Create a new Seurat object with the aggregated counts
    seurat_obj_aggregated <- CreateSeuratObject(
      counts = counts_matrix_aggregated,
      meta.data = seurat_obj_subset@meta.data
    )
    
    # Transfer additional assay data if necessary
    # For example, if you have normalized data, variable features, etc.
    # This step can be more complex depending on your specific data and requirements
    
    # Replace the original Seurat object with the aggregated one
    seurat_objects[[name]] <- seurat_obj_aggregated
    message(paste("Conversion and aggregation completed for", name))
    
  } else {
    message(paste(name, "already uses gene symbols. No conversion needed."))
  }
}

Converting Ensembl IDs to gene symbols in SCPCL000001

'select()' returned 1:many mapping between keys and columns

Warning message:
"The `slot` argument of `GetAssayData()` is deprecated as of SeuratObject 5.0.0.
i Please use the `layer` argument instead."
Warning message:
"Data is of class matrix. Coercing to dgCMatrix."
Conversion and aggregation completed for SCPCL000001

Converting Ensembl IDs to gene symbols in SCPCL000002

'select()' returned 1:many mapping between keys and columns



In [ ]:
lapply(seurat_objects, function(obj) {
  sum(duplicated(rownames(obj)))
})

In [ ]:
combined <- merge(seurat_objects[[1]], y = seurat_objects[2:length(selected_list)], 
                 add.cell.ids = names(seurat_objects))

In [ ]:

# Run Harmony integration
combined <- RunHarmony(
  object = combined,
  group.by.vars = "sample_name", 
  assay.use = "RNA",
  reduction.use = "pca",
  dims.use = 1:20,
  verbose = TRUE
)

# Run UMAP on harmony embeddings
combined <- RunUMAP(combined, reduction = "harmony", dims = 1:20)
saveRDS(combined, file = "./out/reference_integrated_harmony.rds")